https://www.kaggle.com/datasets/purusinghvi/email-spam-classification-dataset?resource=download

Nama              : Geraldy Yobel Primandaru

Username dicoding :geraldyyobel

<style>
table, tr, td {
  border: 1px solid black;
  border-radius: 10px;
}

</style>
<table>
  <tr>
    <th></th>
    <th>Deskripsi</th>
  </tr>
  <tr>
    <td>Dataset</td>
    <td>https://www.kaggle.com/datasets/purusinghvi/email-spam-classification-dataset?resource=download</td>
  </tr>
  <tr>
    <td>Masalah</td>
    <td>masalah utama adalah memastikan bahwa model klasifikasi email dapat membedakan dengan tepat antara email yang merupakan spam dan yang bukan spam. Hubungan antara deskripsi masalah dan dataset ini adalah bahwa dataset menyediakan informasi yang diperlukan untuk melatih model klasifikasi yang dapat membedakan antara email yang sah dan email spam. Dengan menggunakan dataset ini, kita dapat membangun model yang memanfaatkan pola-pola dalam teks email untuk mengidentifikasi ciri-ciri yang membedakan antara email spam dan non-spam. Melalui proses ini, kita dapat mengatasi masalah spam dan meningkatkan pengalaman pengguna dalam mengelola kotak masuk email mereka.</td>
  </tr>
  <tr>
    <td>Solusi Machine Learning</td>
    <td>membangun sebuah machine learning pipeline menggunakan TensorFlow Extended (TFX) untuk melakukan klasifikasi email spam berdasarkan dataset yang diberikan. Dengan melakukan klasifikasi, user dapat menyaring email secara otomatis yang terdeteksi sebagai spam, sehingga dapat meningkatkan produktivitas ketika bekerja. Secara keseluruhan, solusi model Spam-Detection ini memberikan manfaat bagi pengguna dalam hal efisiensi dan keamanan.</td>
  </tr>
  <tr>
    <td>Metode Pengolahan</td>
    <td>metode normalisasi menjadi huruf kecil menggunakan fungsi tf.strings.lower() dari TensorFlow. Ini dilakukan untuk memastikan konsistensi dalam representasi teks, sehingga kata-kata yang sama dengan huruf besar atau huruf kecil dapat dianggap setara. kemudian Tanda baca dihilangkan dari teks email menggunakan pendekatan sederhana dengan tidak menyertakan tanda baca saat normalisasi.</td>
  </tr>
  <tr>
    <td>Arsitektur Model</td>
    <td>1. Normalisasi Teks, 2. Layer Embeding, 3. Layer Conv1D, 4. Dense Layer, 5. Output Layer</td>
  </tr>
  <tr>
    <td>Metrik Evaluasi</td>
    <td>1. ExampleCount dan 2. AUC, 3. FalsePositive, 4. TruePositive, 5. FalseNegative, 6. TrueNegatives, 7. BinaryAccuracy</td>
  </tr>
  <tr>
    <td>Performa Model</td>
    <td>1. ExampleCount: Metrik ini menyajikan jumlah total contoh yang dievaluasi selama proses evaluasi model. Pada kasus ini, terdapat 112 contoh yang dievaluasi. <p>
    2. AUC (Area Under the Curve): Metrik ini mengukur performa model dalam memisahkan kelas positif dan negatif. Nilai AUC berada dalam kisaran 0 hingga 1, di mana nilai yang lebih tinggi menunjukkan performa yang lebih baik. AUC memiliki nilai sebesar 0.97259, yang menunjukkan bahwa model memiliki kinerja yang sangat baik dalam membedakan antara email spam dan bukan spam. <p>
    3. False Positives: Ini adalah jumlah contoh yang salah diklasifikasikan sebagai positif (spam) oleh model, padahal sebenarnya bukan spam. Dalam kasus ini, terdapat 3 contoh yang salah diklasifikasikan sebagai spam. <p>
    4. True Positives: Ini adalah jumlah contoh yang benar diklasifikasikan sebagai positif (spam) oleh model.  Dalam kasus ini, terdapat 60 contoh yang benar diklasifikasikan sebagai spam. <p>
    5. False Negatives: Ini adalah jumlah contoh yang salah diklasifikasikan sebagai negatif (bukan spam) oleh model, padahal sebenarnya spam. Dalam kasus ini, terdapat 8 contoh yang salah diklasifikasikan sebagai bukan spam. <p>
    6. True Negatives: Ini adalah jumlah contoh yang benar diklasifikasikan sebagai negatif (bukan spam) oleh model. Dalam kasus ini, terdapat 41 contoh yang benar diklasifikasikan sebagai bukan spam. <p>
    7. Binary Accuracy: Metrik ini mengukur akurasi keseluruhan model dalam mengklasifikasikan contoh sebagai positif (spam) atau negatif (bukan spam). Ini dihitung sebagai rasio dari jumlah prediksi yang benar (True Positives + True Negatives) dibagi dengan jumlah total contoh. Dalam kasus ini, nilai akurasi yang diberikan adalah 0.9018, yang menunjukkan bahwa model memiliki akurasi sebesar 90.18% dalam memprediksi apakah sebuah email adalah spam atau bukan spam.

  </tr>
</table>

Machine Learning Pipeline

In [24]:
# !pip install -U tfx
# # !pip install jupyter scikit-learn tensorflow tfx
# !pip install jupyter scikit-learn tensorflow tfx==1.11.0 flask joblib

In [25]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [26]:
#import library
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd
import tensorflow_transform as tft
from tensorflow.keras import layers
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

## Set Variable

In [27]:
PIPELINE_NAME = "sarcasm-pipeline"
SCHEMA_PIPELINE_NAME = "sarcasm-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [28]:
# data_path = pd.read_csv("/content/data.csv")
data_path = pd.read_csv("combined_data.csv")
dataa = 'data'
import csv
import sys

# Set batas ukuran bidang yang lebih besar
csv.field_size_limit(sys.maxsize)

# Lakukan parsing CSV seperti biasa
with open('/content/combined_data.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Lakukan sesuatu dengan setiap baris
        pass

# data_path = data_path.rename(columns={'is_sarcastic':'is_sarcastic'})
data_path = data_path.rename(columns={'label':'label'})
# data_path.to_csv(os.path.join(dataa, "data.csv"), index = False)
data_path.to_csv(os.path.join(dataa, "combined_data.csv"), index = False)

In [29]:
DATA_ROOT = "data"

In [30]:
data_path

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
541,1,while we may have high expectations of our ass...
542,1,does size matter' escapenumber of women said t...
543,0,what about using grass escapenumber combined w...
544,1,"the get me thru newslettervenita king "" the ge..."


In [31]:
DATA_ROOT

'data'

In [32]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

In [33]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [34]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Data Validation

In [35]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)


interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [36]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [37]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"]
)


interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [38]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'text',BYTES,required,,-


In [39]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [40]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

In [41]:
TRANSFORM_MODULE_FILE = "sarcasm_transform.py"

In [42]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf


#LABEL_KEY = "is_sarcastic"
#FEATURE_KEY = "headline"
LABEL_KEY = "label"
FEATURE_KEY = "text"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Writing sarcasm_transform.py


In [43]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Latihan Pembuatan Komponen Pengembangan dan Validasi Model dalam Machine Learning Pipeline

## Membuat Tahapan Pengembangan Model

### Training function

In [44]:
TRAINER_MODULE_FILE = "sarcasm_trainer.py"

In [45]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

#LABEL_KEY = "is_sarcastic"
#FEATURE_KEY = "headline"
LABEL_KEY = "label"
FEATURE_KEY = "text"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
            tf_transform_output,
            num_epochs,
            batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset


# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing sarcasm_trainer.py


### Component Trainer

In [46]:
from tfx.proto import trainer_pb2

trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088  


Epoch 1: val_binary_accuracy improved from -inf to 0.90179, saving model to pipelines/sarcasm-pipeline/Trainer/model/6/Format-Serving
1000/1000 [==============================] - 9s 7ms/step - loss: 0.2412 - binary_accuracy: 0.8714 - val_loss: 0.2703 - val_binary_accuracy: 0.9018


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Membuat Tahapan Analisis dan Validasi Model

### Component Resolver

In [47]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Component Evaluator

In [48]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]

)

In [49]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Result: interactive visualization

In [50]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

# Latihan Model Deployment

## Component Pusher

In [51]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/sarchasm-detection-model'))

)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [52]:
# !nohup tensorflow_model_server --port=1007 --model_name=sarchasm-detection-model --model_base_path=/path/to/your/model > server.log 2>&1 &
!nohup tensorflow_model_server --port=4040 --model_name=sarchasm-detection-model --model_base_path=/path/to/your/model > server.log 2>&1 &


In [53]:
# get_ipython().system_raw('./ngrok http 1007 &')
get_ipython().system_raw('./ngrok http 4040 &')


In [54]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [55]:
! pip install pyngrok

! ngrok authtoken 2fRtsqKKg97OYHEgXIiUCdRkbCR_2f7XjWBpNyMXXm9rKFqeousr_2fRtsuHjaywC5zuliUJqL6UqjBe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [56]:
!ngrok config add-authtoken 2fRtsqKKg97OYHEgXIiUCdRkbCR_2f7XjWBpNyMXXm9rKFqeo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [57]:
!authtoken: 2fRtsqKKg97OYHEgXIiUCdRkbCR_2f7XjWBpNyMXXm9rKFqeo

/bin/bash: line 1: authtoken:: command not found


In [58]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('tensorboard --logdir /content/trainingdata/objectdetection/ckpt_output/trainingImatges/ --host 0.0.0.0 --port 6006 &')

get_ipython().system_raw('./ngrok http 6006 &')

--2024-04-27 22:59:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  17.7MB/s    in 0.7s    

2024-04-27 22:59:50 (17.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [59]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
 "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [60]:
from pyngrok import ngrok

# Start the HTTP tunnel using ngrok
http_tunnel = ngrok.connect(addr="4040", proto="http")

# Print the public URL of the tunnel
print("Public URL:", http_tunnel.public_url)


Public URL: https://4fad-35-197-107-238.ngrok-free.app


In [61]:
# import requests
# from pprint import PrettyPrinter

# pp = PrettyPrinter()

# try:
#     response = requests.get("https://428f-34-29-135-154.ngrok-free.app")
#     response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
#     if response.text:  # Check if the response body is not empty
#         pp.pprint(response.json())
#     else:
#         print("Response body is empty.")
# except requests.exceptions.RequestException as e:
#     print("Error:", e)


In [62]:
!zip -r /content/serving_model_dir.zip /content/serving_model_dir
!zip -r /content/pipelines.zip /content/pipelines
!zip -r /content/data.zip /content/data

  adding: content/serving_model_dir/ (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/ (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/ (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/assets/ (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/variables/ (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/variables/variables.data-00000-of-00001 (deflated 56%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/variables/variables.index (deflated 59%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/saved_model.pb (deflated 80%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/fingerprint.pb (stored 0%)
  adding: content/serving_model_dir/sarchasm-detection-model/1714258766/keras_metadata.pb (deflated 85%)
  adding: content/pipelines/ (stored 0%)
  adding: content/

In [63]:
from google.colab import files
files.download('/content/serving_model_dir')
files.download('/content/pipelines')
files.download('/content/data')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
!pip freeze > requirements.txt